In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
import torchvision.models as models
from PIL import Image
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb

from functools import partial
from collections import OrderedDict

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import time

hidden_width = 32
hidden_nblocks = 4
train_max_epoch = 50

data_root = "geomorph_data"
n_channels = len(os.listdir(data_root))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
L2_param = 1e-5

In [2]:
class mlp(nn.Module):
        def __init__(self, input_size, output_size = 1, hidden_width = 20, hidden_nblocks = 2):
            super(mlp, self).__init__()
            self.input_size = input_size
            self.output_size = output_size
            
            self.hidden_width = hidden_width
            self.hidden_nblocks = hidden_nblocks
            
            self.fc1 = nn.Linear(self.input_size, self.hidden_width)
            self.fc2 = nn.Linear(self.hidden_width,self.hidden_width)
            self.fc3 = nn.Linear(self.hidden_width, self.output_size)
            
            self.relu = torch.nn.ReLU()
            self.sigmoid = torch.nn.Sigmoid() ## sigmoid for multi-label, softmax for multi-class (mutually exclusive)
            
            self.dropout = nn.Dropout(0.25)
            
        def forward(self, x, film_params):
            out = self.fc1(x)
            out = self.relu(out)
            
            
            for i in range(self.hidden_nblocks):
                out = self.fc2(out)
                
                # ------- film layer -----------
                start = i * hidden_width * 2
                mid = start + hidden_width
                end = mid + hidden_width
                
                gamma = film_params[:, start : mid]
                beta = film_params[:, mid : end]
                
#                 print(out.shape)
#                 print(gamma.shape)
#                 print(beta.shape)
                
                out = out * gamma
                out += beta
                # ------- film layer -----------
                # out = self.dropout(out)
                out = self.relu(out)
            
            out = self.fc3(out)
            out = self.sigmoid(out)
            return out

## Dataset

In [3]:
# scaler = MinMaxScaler()
def normalize(values):
    # zero mean, unit variance
    return (values-values.mean())/values.std()

def normalize_maxmin(values):
    # range from 0 to 1
    (values-values.min())/(values.max()-values.min())

def preprocess_df(df):
    # convert timecodes to year and month columns
    datetimes = pd.to_datetime(df['time'])
    df['month'] = datetimes.dt.month
    df['year'] = datetimes.dt.year

    df['month_cyclic'] = 7 - abs(df['month'] - 7)

    df['lat_norm'] = normalize(df['latitude'])
    df['lng_norm'] = normalize(df['longitude'])
    df['depth_norm'] = normalize(df['depth'])
    df['year_norm'] = normalize(df['year'])
    df['month_cyclic_norm'] = normalize(df['month_cyclic'])

df = pd.read_csv('data_stephen_fix_header.csv', header=[0])
preprocess_df(df)
    
print(df.shape[0])
print(df['borehole'].nunique())
df.head()

2837
566


,latitude,longitude,time,borehole,depth,frozen,cryostructures,visible_ice,ASTM_2488,materials,...,top_of_interval,bottom_of_interval,month,year,month_cyclic,lat_norm,lng_norm,depth_norm,year_norm,month_cyclic_norm
0,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,0.15,0,NaN,NaN,TOPSOIL,Organics,...,0.0,0.3,3,2012,3,1.439692,1.851129,-1.024010,-1.164786,-1.225079
1,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,0.85,1,NaN,Pure ice,ICE,Ice,...,0.3,1.4,3,2012,3,1.439692,1.851129,-0.835753,-1.164786,-1.225079
2,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,1.90,1,Nf,No visible ice,SW-SM,Coarse till,...,1.4,2.4,3,2012,3,1.439692,1.851129,-0.553369,-1.164786,-1.225079
3,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,5.40,1,Nf,No visible ice,GW-GM,Coarse till,...,2.4,8.4,3,2012,3,1.439692,1.851129,0.387914,-1.164786,-1.225079
4,69.16105,-133.08880,2012-03-21T00:00:00Z,0170-1-12,1.20,1,Nf,No visible ice,GP-GM,Coarse till,...,0.0,2.4,3,2012,3,1.437542,1.842296,-0.741625,-1.164786,-1.225079


In [4]:
df2 = df.dropna(subset=['visible_ice'])
df2['visible_ice'].replace(['None'], 'No visible ice', regex=True, inplace=True)
df2.tail()

c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,latitude,longitude,time,borehole,depth,frozen,cryostructures,visible_ice,ASTM_2488,materials,...,top_of_interval,bottom_of_interval,month,year,month_cyclic,lat_norm,lng_norm,depth_norm,year_norm,month_cyclic_norm
2832,68.38262,-133.71211,2013-04-27T00:00:00Z,W14103137-S6-BH15,5.45,1,Nf/Nbn,No visible ice,NaN,Till,...,2.7,8.2,4,2013,4,-1.498903,-0.938394,0.401360,-0.318997,0.736292
2833,68.38262,-133.71211,2013-04-27T00:00:00Z,W14103137-S6-BH15,8.65,1,Nbn,No visible ice,NaN,Till,...,8.2,9.1,4,2013,4,-1.498903,-0.938394,1.261961,-0.318997,0.736292
2834,68.38386,-133.70967,2013-04-27T00:00:00Z,W14103137-S6-BH16,0.05,0,NaN,No visible ice,ORGANICS,Organics,...,0.0,0.1,4,2013,4,-1.494226,-0.927509,-1.050903,-0.318997,0.736292
2835,68.38386,-133.70967,2013-04-27T00:00:00Z,W14103137-S6-BH16,0.35,0,NaN,No visible ice,NaN,Till,...,0.1,0.6,4,2013,4,-1.494226,-0.927509,-0.970222,-0.318997,0.736292
2836,68.38386,-133.70967,2013-04-27T00:00:00Z,W14103137-S6-BH16,4.85,1,Nbn,No visible ice,NaN,Till,...,0.6,9.1,4,2013,4,-1.494226,-0.927509,0.239998,-0.318997,0.736292


In [5]:
# check None values have been replaced
len(df2[df2['visible_ice'] == 'None'])

0

In [6]:
visible_ice = pd.get_dummies(df2.visible_ice)
bin_visible_ice = (~visible_ice['No visible ice'].astype('bool')).astype('int')
bin_visible_ice.value_counts()

1    1796
0     956
Name: No visible ice, dtype: int64

In [7]:
df2['visible_ice'] = bin_visible_ice

<ipython-input-7-697191a84568>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['visible_ice'] = bin_visible_ice


In [8]:
df2.head()

,latitude,longitude,time,borehole,depth,frozen,cryostructures,visible_ice,ASTM_2488,materials,...,top_of_interval,bottom_of_interval,month,year,month_cyclic,lat_norm,lng_norm,depth_norm,year_norm,month_cyclic_norm
1,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,0.85,1,NaN,1,ICE,Ice,...,0.3,1.4,3,2012,3,1.439692,1.851129,-0.835753,-1.164786,-1.225079
2,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,1.90,1,Nf,0,SW-SM,Coarse till,...,1.4,2.4,3,2012,3,1.439692,1.851129,-0.553369,-1.164786,-1.225079
3,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,5.40,1,Nf,0,GW-GM,Coarse till,...,2.4,8.4,3,2012,3,1.439692,1.851129,0.387914,-1.164786,-1.225079
4,69.16105,-133.08880,2012-03-21T00:00:00Z,0170-1-12,1.20,1,Nf,0,GP-GM,Coarse till,...,0.0,2.4,3,2012,3,1.437542,1.842296,-0.741625,-1.164786,-1.225079
5,69.16105,-133.08880,2012-03-21T00:00:00Z,0170-1-12,3.95,1,Nf,0,SM,Sand,...,2.4,5.5,3,2012,3,1.437542,1.842296,-0.002046,-1.164786,-1.225079


In [9]:
df3 = df.dropna(subset=['materials'])
df3['materials'].replace(['ICE'], 'Ice', regex=True, inplace=True)
df3['materials'].replace(['ice'], 'Ice', regex=True, inplace=True)

In [10]:
dm_materials = pd.get_dummies(df3.materials)
df3['material_ice'] = dm_materials['Ice']

In [11]:
m = nn.Dropout(p=0.2)


class Geo90Dataset(Dataset):
    def __init__(self, data_root, df, base_lat, base_lng, chip_size=32, label_name = 'frozen'):
        
        self.base_lat = base_lat
        self.base_lng = base_lng
        
        self.df = df
        
        self.chip_size = chip_size
        self.label_name = label_name
        
        self.trans = transforms.ToTensor()
        
        self.n_channels = n_channels
        self.preloaded = torch.ones(self.n_channels, 6000, 6000)
        
        for i, file in enumerate(os.listdir(data_root)):
            # name = file.split('_')[0]
            # print(name)
            self.preloaded[i] = self.trans(Image.open(data_root + os.path.sep + file))
        
        print('Dataset initialized')
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        bh_id = row.at['borehole']
        lat = row.at['latitude']
        lng = row.at['longitude']
        

        pixel_len = 5/6000
        

        lat_index_start = np.round((self.base_lat - lat) / pixel_len - self.chip_size/2).astype(int)
        lat_index_end = lat_index_start + self.chip_size
        
        lng_index_start = np.round((lng - self.base_lng) / pixel_len - self.chip_size/2).astype(int)
        lng_index_end = lng_index_start + self.chip_size
        
        image = self.preloaded[:, lat_index_start:lat_index_end,lng_index_start:lng_index_end]
        
        
        # surface = torch.tensor(row.filter(['depth'])).float()
        surface = torch.tensor(row.filter(['depth_norm', 'month_cyclic_norm', 'lat_norm', 'lng_norm', 'year_norm'])).float()
        
        frozen = torch.tensor(row.at['frozen']).float()
        
        visible_ice = torch.tensor(row.at['visible_ice']).float()
        
        # material_ice = torch.tensor(row.at['material_ice']).float()
        
        return {'image': image, 'surface_data': surface, 'frozen': frozen,  'visible_ice': visible_ice} #'material_ice': material_ice}

## Train model

## FiLM

In [12]:
n_film_params = hidden_width * hidden_nblocks * 2

# generator = resnet18(n_channels, n_film_params)

def train_model(trainloader, testloader, label_name, print_epochs = False, loss_fn = torch.nn.BCELoss()):
    # loss: binary cross entropy

    generator = models.resnet18()
    generator.fc = nn.Linear(512, n_film_params)
    generator.conv1 = nn.Conv2d(n_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    
    gen_model = generator

    # print(gen_model)

    input_size = list(full_dataset[0]['surface_data'].size())
    net_model = mlp(input_size[0],1, hidden_width, hidden_nblocks).to(device)
    
    gen_optimizer = torch.optim.Adam(gen_model.parameters(), weight_decay = L2_param)
    net_optimizer = torch.optim.Adam(net_model.parameters(), weight_decay = L2_param)
    
    gen_model.to(device)
    net_model.to(device)

    # --------- check back propagation ----------- -
    # net_model.fc1.weight.register_hook(lambda x: print('grad accumulated in mlp fc1'))
    # gen_first_layer = gen_model.encoder.blocks[0].blocks[0].blocks[0].conv
    # gen_first_layer.weight.register_hook(lambda x: print('grad accumulated in resnet first layer'))

    epoch_loss = np.zeros([train_max_epoch, 2])
    for epoch in range(train_max_epoch):  # loop over the dataset multiple times

        # ------------ train -----------------
        gen_model.train()
        net_model.train()
        running_loss_sum = 0.0
        for i, data in enumerate(trainloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            images, surface_data, labels = data['image'].to(device), data['surface_data'].to(device), data[label_name].to(device)

            
            gen_params = gen_model(images)
            predicted = net_model(surface_data, gen_params)
            predicted = torch.squeeze(predicted)
            loss = loss_fn(predicted, labels)

            gen_optimizer.zero_grad()
            net_optimizer.zero_grad()

            loss.backward()

            gen_optimizer.step()
            net_optimizer.step()

            running_loss_sum += loss.item()

        # ----------- get validation loss for current epoch --------------
        gen_model.eval()
        net_model.eval()
        validation_loss_sum = 0.0
        for i, data in enumerate(testloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            images, surface_data, labels = data['image'].to(device), data['surface_data'].to(device), data[label_name].to(device)

            # TODO: exammine film_params gradients / readup pytorch
            gen_params = gen_model(images)
            predicted = net_model(surface_data, gen_params)
            predicted = torch.squeeze(predicted)
            loss = loss_fn(predicted, labels)
            validation_loss_sum += loss.item()

        # ---------------- print statistics ------------------------

        running_loss = running_loss_sum / len(trainloader)
        validation_loss = validation_loss_sum / len(testloader)
        epoch_loss[epoch, :] =  [running_loss, validation_loss]
        
        if print_epochs:
            print('epoch %2d: running loss: %.5f, validation loss: %.5f' %
                          (epoch + 1, running_loss, validation_loss))

        torch.save(gen_model.state_dict(), os.path.join('mlp-resnet-models/', 'gen-epoch-{}.pt'.format(epoch+1)))
        torch.save(net_model.state_dict(), os.path.join('mlp-resnet-models/', 'net-epoch-{}.pt'.format(epoch+1)))

    if print_epochs:
        print('Finished Training')
    
    return epoch_loss

## Test model

def test_model(epoch_loss, label_name, print_model_epoch = False):
    
    # ------ select model ---------
    ind = np.argmin(epoch_loss[:, 1])
    
    generator = models.resnet18()
    generator.fc = nn.Linear(512, n_film_params)
    generator.conv1 = nn.Conv2d(n_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    
    gen_model = generator

    input_size = list(full_dataset[0]['surface_data'].size())
    net_model = mlp(input_size[0],1, hidden_width, hidden_nblocks)

    gen_model.load_state_dict(torch.load('mlp-resnet-models/gen-epoch-{}.pt'.format(ind+1)))
    net_model.load_state_dict(torch.load('mlp-resnet-models/net-epoch-{}.pt'.format(ind+1)))
    
    gen_model.to(device)
    net_model.to(device)
    
    if print_model_epoch:
        print("epoch {} model selected".format(ind+1))
    
    # evaluate model on test set
    gen_model.eval()
    net_model.eval()
    with torch.no_grad():
        y_test = []
        y_pred = []
        for i, data in enumerate(testloader, 0):
            images, surface_data, labels = data['image'].to(device), data['surface_data'].to(device), data[label_name].to(device)

            # y_test.append(label.numpy().list())
            # print(label.shape)
            # print(images.shape)

            gen_params = gen_model(images)
            predicted = net_model(surface_data, gen_params)
            predicted = torch.squeeze(predicted)

            predicted = torch.round(predicted)
            # print(predicted.shape)
            lb = labels.tolist()
            pr = predicted.tolist()
            y_test.extend(lb)
            y_pred.extend(pr)
    
    arr_accuracy = accuracy_score(y_test, y_pred)
    scores = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=0)
    return arr_accuracy, scores


#     print(confusion_matrix(y_test,y_pred))
#     print(classification_report(y_test,y_pred))
#     print(accuracy_score(y_test, y_pred))

## Pure MLP

## MLP

In [13]:
class mlp_pure(nn.Module):
        def __init__(self, input_size, output_size):
            super(mlp_pure, self).__init__()
            self.input_size = input_size
            self.output_size = output_size
            
            self.hidden_size = hidden_width
            self.hidden_nblocks = hidden_nblocks
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size,self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, self.output_size)
            
            self.relu = torch.nn.ReLU()
            self.sigmoid = torch.nn.Sigmoid() ## sigmoid for multi-label, softmax for multi-class (mutually exclusive)
            
        def forward(self, x):
            out = self.fc1(x)
            out = self.relu(out)
            # print(out.shape)
            
            for i in range(self.hidden_nblocks):
                out = self.fc2(out)
                out = self.relu(out)
            
            out = self.fc3(out)
            out = self.sigmoid(out)
            return out

def train_mlp(trainloader, testloader, label_name, print_epochs = False, loss_fn = torch.nn.BCELoss()):
    input_size = list(full_dataset[0]['surface_data'].size())
    surface_model = mlp_pure(input_size[0],1)
    
    surface_model.to(device)
    
    optimizer = torch.optim.Adam(surface_model.parameters(), weight_decay = L2_param)

    epoch_loss = np.zeros([train_max_epoch, 2])
    for epoch in range(train_max_epoch):  # loop over the dataset multiple times

        surface_model.train()
        running_loss_sum = 0.0
        for i, data in enumerate(trainloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            surface_data, labels = data['surface_data'].to(device), data[label_name].to(device)

            predicted = surface_model(surface_data)

            loss = loss_fn(predicted.squeeze(), labels)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            running_loss_sum += loss.item()

        # ----------- get validation loss for current epoch --------------
        surface_model.eval()
        validation_loss_sum = 0.0
        for i, data in enumerate(testloader, 0): # loop over each sample

            surface_data, labels = data['surface_data'].to(device), data[label_name].to(device)

            predicted = surface_model(surface_data)

            loss = loss_fn(predicted.squeeze(), labels)

            validation_loss_sum += loss.item()

        # ---------------- print statistics ------------------------

        running_loss = running_loss_sum / len(trainloader)
        validation_loss = validation_loss_sum / len(testloader)
        epoch_loss[epoch, :] =  [running_loss, validation_loss]
        
        if print_epochs:
            print('epoch %2d: running loss: %.5f, validation loss: %.5f' %
                          (epoch + 1, running_loss, validation_loss))

        torch.save(surface_model.state_dict(), os.path.join('mlp-models/', 'epoch-{}.pt'.format(epoch+1)))
    
    if print_epochs:
        print('Finished Training')
        
    return epoch_loss
        
def test_mlp(epoch_loss, label_name, print_model_epoch = False):
    
    # ------ select model ---------
    ind = np.argmin(epoch_loss[:, 1])
    
    input_size = list(full_dataset[0]['surface_data'].size())
    
    surface_model = mlp_pure(input_size[0],1)
    surface_model.load_state_dict(torch.load('mlp-models/epoch-{}.pt'.format(ind+1)))
    
    surface_model.to(device)
    
    if print_model_epoch:
        print("epoch {} model selected".format(ind+1))
    
    # evaluate model on test set
    surface_model.eval()

    with torch.no_grad():
        y_test = []
        y_pred = []
        for i, data in enumerate(testloader, 0):
            surface_data, labels = data['surface_data'].to(device), data[label_name].to(device)

            # y_test.append(label.numpy().list())
            # print(label.shape)
            # print(images.shape)

            output = surface_model(surface_data)
            predicted = torch.round(output)
            # print(predicted.shape)
            lb = labels.tolist()
            pr = predicted.tolist()
            y_test.extend(lb)
            y_pred.extend(pr)
    
    arr_accuracy = accuracy_score(y_test, y_pred)
    scores = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=0)
    return arr_accuracy, scores


## Multiple Runs

In [14]:
base_lat = 70
base_lng = -135

loaded_dataset = Geo90Dataset(data_root, df2, base_lat, base_lng, chip_size = 32)

# image = full_dataset[0]['image']
# n_channels = list(image.shape)[0]

# for data in full_dataset:
#     image = data['image']
#     for i in range(n_channels):
#         channel = image[i]
#         ind = (channel == -9999)
# #         mean_val = torch.mean(channel[~ind])
#         channel[ind] = 0
#         data['image'][i] = channel
        

Dataset initialized


In [16]:
valid_ind = []
for i, data in enumerate(loaded_dataset):
    image = data['image']
    ind  = (image == -9999)
    if ~torch.any(ind):
        valid_ind.append(i)

print(len(loaded_dataset))
print(len(valid_ind))

full_dataset = torch.utils.data.Subset(loaded_dataset, valid_ind)

2752
2717


In [17]:


train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

# batchsize can cause error when last leftover batchsize is 1, batchnorm cannot function on 1 sample data
batchsize = 20
while(train_size % batchsize == 1):
    batchsize+=1
print(batchsize)

train_data, test_data = torch.utils.data.random_split(full_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

trainloader = DataLoader(train_data, batch_size=batchsize, shuffle=True)
testloader = DataLoader(test_data, batch_size=batchsize, shuffle=True)

20


## Scale of image chips

In [ ]:
# image = full_dataset[0]['image']
# n_samples = len(train_data)
# n_channels = list(image.shape)[0]

# scalers = []
# for i in range(n_channels):
    
#     scaler = StandardScaler()
#     X = torch.empty((n_samples, full_dataset.chip_size, full_dataset.chip_size))
    
#     for j, data in enumerate(train_data):
#         #print(data['image'][i].shape)
#         # print(X[j].shape)
#         X[j] = data['image'][i]
#     X = torch.reshape(X, (-1,1))
#     scaler.fit(X)
#     scalers.append(scaler)
    
#     def scale_data(subset):
#         for data in subset:
#             X = data['image'][i]
#             X_flat = torch.reshape(X, (-1,1))
#             X_trans = scaler.transform(X_flat)
#             data['image'][i] = torch.reshape(torch.Tensor(X_trans), (full_dataset.chip_size, full_dataset.chip_size))
    
#     scale_data(train_data)
#     scale_data(test_data)
#     print("Channel {} scaled.".format(i))

In [ ]:
max_iterations = 10
results = np.zeros([max_iterations, 9])
# trainloader, testloader = prepare_dataloader(full_dataset)

labelName = 'visible_ice'

for it in range(max_iterations):
    start = time.time()
    
    # mlp
#     epoch_loss_mlp = train_mlp(trainloader,testloader, labelName, print_epochs = True)
#     acc, scores = test_mlp(epoch_loss_mlp, labelName, print_model_epoch = True)
    
    #     # ------- mlp-resnet film 
    epoch_loss = train_model(trainloader, testloader, labelName, print_epochs=True)
    acc, scores = test_model(epoch_loss, labelName, print_model_epoch = True)
    
    results[it, 0:2] = scores[0]
    results[it, 2:4] = scores[1]
    results[it, 4:6] = scores[2]
    results[it, 6:8] = scores[3]
    results[it, 8] = acc 
    
    end = time.time()
    
    print('iteration {} elapsed time: {}, accuracy : {}'.format(it+1, end-start, acc))


c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch  1: running loss: 0.66140, validation loss: 0.65125
epoch  2: running loss: 0.63698, validation loss: 0.66321
epoch  3: running loss: 0.63809, validation loss: 0.67553
epoch  4: running loss: 0.62974, validation loss: 0.65741
epoch  5: running loss: 0.62934, validation loss: 0.66083
epoch  6: running loss: 0.63097, validation loss: 0.66313
epoch  7: running loss: 0.63184, validation loss: 0.65475
epoch  8: running loss: 0.62344, validation loss: 0.65215
epoch  9: running loss: 0.62377, validation loss: 0.72525
epoch 10: running loss: 0.62740, validation loss: 0.66495
epoch 11: running loss: 0.62036, validation loss: 0.64918
epoch 12: running loss: 0.61020, validation loss: 0.66288
epoch 13: running loss: 0.61206, validation loss: 0.66076
epoch 14: running loss: 0.61214, validation loss: 0.66032
epoch 15: running loss: 0.61369, validation loss: 0.65803
epoch 16: running loss: 0.61144, validation loss: 0.65248
epoch 17: running loss: 0.59941, validation loss: 0.65861
epoch 18: runn

In [ ]:
def display_table(scores):
    df = np.reshape(scores, [2,4], order ='F')
    df = pd.DataFrame(df)
    
    # df.style.set_table_attributes("style='display:inline'").set_caption(mode)
    
    df.columns = ['precision', 'recall', 'f1', 'support']
    # df.index = ['unfrozen', 'frozen']
    # df.index = ['Visible ice', 'No visible ice']
    
    display(df)
    
def display_results(results):
    mean = np.mean(results, axis=0)
    std = np.std(results, axis=0)
    
    print("mean")
    display_table(mean[0:8])
    
    print("std")
    display_table(std[0:8])
    
    print("Accuracy mean: {}, std: {}".format(mean[8], std[8]))

display_results(results)